**BART TECHNIQUE WITH ROUGE SCORE**

In [ ]:
!pip install transformers
!pip install datasets
!pip install rouge-score


INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=a40265a06c4f2399e2c83b51f8f4e6eb2b270067223b222eaf452c7c8391d277
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


**Load BART Model and Tokenizer**

In [ ]:
from transformers import BartForConditionalGeneration, BartTokenizer

# Load the pre-trained BART model and tokenizer
model_name = 'facebook/bart-large-cnn'
tokenizer = BartTokenizer.from_pretrained(model_name)
model = BartForConditionalGeneration.from_pretrained(model_name)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

**Prepare Input Data**

In [ ]:
text = """Natural Language Processing (NLP) is a fascinating and rapidly evolving field at the intersection of artificial intelligence, linguistics, and computer science. It focuses on enabling machines to understand, interpret, and generate human language in a way that is both meaningful and useful.Natural Language Processing is a dynamic field with the potential to transform how we interact with technology and communicate with each other. As research continues to progress, the integration of NLP into various sectors will likely enhance efficiency, accessibility, and user experience across numerous applications. The challenges ahead present opportunities for innovation, making NLP a key area of interest for researchers, developers, and businesses alike."""


**Tokenize and Summarize**

In [ ]:
# Tokenize the input text
inputs = tokenizer([text], max_length=1024, return_tensors='pt', truncation=True)

# Generate summary
summary_ids = model.generate(inputs['input_ids'], max_length=150, min_length=40, length_penalty=2.0, num_beams=4, early_stopping=True)

# Decode the summary
summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

print("Summary:")
print(summary)


Summary:
Natural Language Processing (NLP) is a rapidly evolving field at the intersection of artificial intelligence, linguistics, and computer science. It focuses on enabling machines to understand, interpret, and generate human language in a way that is both meaningful and useful.


**Evaluate with ROUGE Score**

In [ ]:
from rouge_score import rouge_scorer

# Reference summary for comparison (you can replace this with your actual summary)
reference_summary = "Natural Language Processing (NLP) is a fascinating and rapidly evolving field at the intersection of artificial intelligence, linguistics, and computer science. It focuses on enabling machines to understand, interpret, and generate human language in a way that is both meaningful and useful.Natural Language Processing is a dynamic field with the potential to transform how we interact with technology and communicate with each other. As research continues to progress, the integration of NLP into various sectors will likely enhance efficiency, accessibility, and user experience across numerous applications. The challenges ahead present opportunities for innovation, making NLP a key area of interest for researchers, developers, and businesses alike."

# Initialize the scorer
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

# Calculate ROUGE scores
scores = scorer.score(reference_summary, summary)

print("ROUGE Scores:")
print(scores)


ROUGE Scores:
{'rouge1': Score(precision=1.0, recall=0.37383177570093457, fmeasure=0.5442176870748299), 'rouge2': Score(precision=0.9743589743589743, recall=0.3584905660377358, fmeasure=0.5241379310344827), 'rougeL': Score(precision=1.0, recall=0.37383177570093457, fmeasure=0.5442176870748299)}


**Putting It All Together**

In [ ]:
def summarize_and_evaluate(text, reference_summary):
    # Tokenize and summarize
    inputs = tokenizer([text], max_length=1024, return_tensors='pt', truncation=True)
    summary_ids = model.generate(inputs['input_ids'], max_length=150, min_length=40, length_penalty=2.0, num_beams=4, early_stopping=True)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

    # Calculate ROUGE scores
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    scores = scorer.score(reference_summary, summary)

    return summary, scores

# Example usage
text = """Your long article or text goes here."""
reference_summary = "Your reference summary goes here."
summary, scores = summarize_and_evaluate(text, reference_summary)

print("Generated Summary:")
print(summary)
print("ROUGE Scores:")
print(scores)


Generated Summary:
Your long article or text goes here. Your long article/text goes to the bottom of the page. For more information, go to CNN.com/soulmatestories.
ROUGE Scores:
{'rouge1': Score(precision=0.11538461538461539, recall=0.6, fmeasure=0.1935483870967742), 'rouge2': Score(precision=0.04, recall=0.25, fmeasure=0.06896551724137932), 'rougeL': Score(precision=0.11538461538461539, recall=0.6, fmeasure=0.1935483870967742)}
